In [1]:
import tweepy
from tweepy import OAuthHandler
import json
import datetime as dt
import time
import os
import sys
import pyodbc
import pandas as pd

In [6]:
conn = pyodbc.connect("DRIVER={SQL Server}; server=.; database=news_crawler1; uid=; pwd=")
c = conn.cursor()

def load_api():
    
    consumer_key = 'HmJLfA9Sjj000twIP5ENmP49x'
    consumer_secret = 'dE6S5p7weInkv9DGRjdWGsW3XMP0nwWuvw6cCxwvUWdDeZPSo0'
    access_token = '1637346992-b0Fx3qXIvLBO3SLBv9ISkEwQLIpuM0HKvNQux1p'
    access_secret = 'EV6kQVcxO3h7S6hhUb6Pz2xNbuiAOvXX4rysRsA0ARnwP'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # load the twitter API via tweepy
    return tweepy.API(auth)

def tweet_search(api, query, max_tweets, max_id, since_id, geocode):
    iddb = 1
    newsPortal = "twitter_db"
    searched_tweets = []
    while len(searched_tweets) < max_tweets:
        remaining_tweets = max_tweets - len(searched_tweets)
        try:
            new_tweets = api.search(q=query, count=remaining_tweets,
                                    since_id=str(since_id),
                                    max_id=str(max_id-1))
            print('found',len(new_tweets),'tweets')
            print(new_tweets)
            for i in range(len(new_tweets)):
#                 print(json.dumps(new_tweets[i]._json))
                iddb += 1
                news_id = new_tweets[i]._json['id_str']
                news_article = new_tweets[i]._json['text']
                favorite_count = new_tweets[i]._json['favorite_count']
                retweet_count = new_tweets[i]._json['retweet_count']
                news_language = new_tweets[i]._json['lang']
                location = new_tweets[i]._json['user']['location']
                userid = new_tweets[i]._json['user']['id']
                isPromo = ""
                created = dt.datetime.now()
                try:
                    link = new_tweets[i]._json['entities']['urls'][0]['url']
                except:
                    link = new_tweets[i]._json['user']['url']
                userName = new_tweets[i]._json['user']['name']
                userScreenName = new_tweets[i]._json['user']['screen_name']
                userLocation = new_tweets[i]._json['user']['location']
                userDesc = new_tweets[i]._json['user']['description']
                userProtected = new_tweets[i]._json['user']['protected']
                userFollowers = new_tweets[i]._json['user']['followers_count']
                userFriends = new_tweets[i]._json['user']['friends_count']
                userFavourites = new_tweets[i]._json['user']['favourites_count']
                userPhotoURL = new_tweets[i]._json['user']['profile_image_url']
                userBannerURL = new_tweets[i]._json['user']['profile_background_image_url']
                tweet_hastags = new_tweets[i]._json['entities']['hashtags']
#######################################################################################################                                                                             
                trigger = 0
                c.execute("SELECT * FROM twitter_tweet WHERE tweet_id=cast(? as varchar)",news_id)
                for row in c:
                    if row[0] is not None:
                        trigger = 1

                if trigger == 0:
                    c.execute("""INSERT INTO twitter_tweet (id, tweet_id, tweet_article, favorite_count, retweet_count,
                                lang, location, userid, link, isPromo, created) VALUES (?,?,?,?,?,?,?,?,?,?,?)""", 
                              (iddb, news_id, news_article, favorite_count, retweet_count,
                                news_language, location, userid, link, isPromo, created))
                    conn.commit()
#######################################################################################################
                trigger_person = 0
                c.execute("SELECT * FROM twitter_person WHERE user_id=cast(? as varchar)",userid)
                for row in c:
                    if row[0] is not None:
                        trigger_person = 1

                if trigger_person == 0:
                    c.execute("""INSERT INTO twitter_person (id, user_id, created, name, 
                                screen_name, location, description, protected, followers, 
                                friends, favourites, photo_URL, banner_URL) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)""", 
                              (iddb, userid, created, userName, userScreenName, userLocation, 
                                userDesc, userProtected, userFollowers, userFriends, userFavourites,
                                userPhotoURL, userBannerURL)
                    conn.commit()
#######################################################################################################
            if not new_tweets:
                print('no tweets found')
                break
            searched_tweets.extend(new_tweets)
            max_id = new_tweets[-1].id
        except tweepy.TweepError:
            print('exception raised, waiting 15 minutes')
            print('(until:', dt.datetime.now()+dt.timedelta(minutes=15), ')')
            time.sleep(15*60)
            break # stop the loop
    return searched_tweets, max_id


def get_tweet_id(api, date='', days_ago=9, query='a'):

    if date:
        # return an ID from the start of the given day
        td = date + dt.timedelta(days=1)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        tweet = api.search(q=query, count=1, until=tweet_date)
    else:
        # return an ID from __ days ago
        td = dt.datetime.now() - dt.timedelta(days=days_ago)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        # get list of up to 10 tweets
        tweet = api.search(q=query, count=10, until=tweet_date)
        print('search limit (start/stop):',tweet[0].created_at)
    
        # return the id of the first tweet in the list
        return tweet[0].id


            
         
def main():
    search_phrases = ['kpu', 'mk','fadli zon','prabowo','amien rais','jokowi', 'pilpres','curang','gugatan']
    time_limit = 1.5                           # runtime limit in hours
    max_tweets = 100                            # number of tweets per search (will be
    min_days_old, max_days_old = 1, 2          # search limits e.g., from 7 to 8
    IND = '-2.548926,118.0148634,4500km'

    for search_phrase in search_phrases:
        print('Search phrase =', search_phrase)
        name = search_phrase.split()[0]
        json_file_root = name + '/'  + name
        read_IDs = False

        if max_days_old - min_days_old == 1:
            d = dt.datetime.now() - dt.timedelta(days=min_days_old)
            day = '{0}-{1:0>2}-{2:0>2}'.format(d.year, d.month, d.day)
        else:
            d1 = dt.datetime.now() - dt.timedelta(days=max_days_old-1)
            d2 = dt.datetime.now() - dt.timedelta(days=min_days_old)
            day = '{0}-{1:0>2}-{2:0>2}_to_{3}-{4:0>2}-{5:0>2}'.format(
                  d1.year, d1.month, d1.day, d2.year, d2.month, d2.day)
        json_file = json_file_root + '_' + day + '.json'
        if os.path.isfile(json_file):
            print('Appending tweets to file named: ',json_file)
            read_IDs = True
            
        api = load_api()
        if read_IDs:
            with open(json_file, 'r') as f:
                lines = f.readlines()
                max_id = json.loads(lines[-1])['id']
                print('Searching from the bottom ID in file')
        else:
            # get the ID of a tweet that is min_days_old
            if min_days_old == 0:
                max_id = -1
            else:
                max_id = get_tweet_id(api, days_ago=(min_days_old-1))

        since_id = get_tweet_id(api, days_ago=(max_days_old-1))
        print('max id (starting point) =', max_id)
        print('since id (ending point) =', since_id)

        ''' tweet gathering loop  '''
        start = dt.datetime.now()
        end = start + dt.timedelta(hours = time_limit)
        count, exitcount = 0, 0
        while dt.datetime.now() < end:
            count += 1
            print('count =',count)
            # collect tweets and update max_id
            tweets, max_id = tweet_search(api, search_phrase, max_tweets,
                                          max_id=max_id, since_id=since_id, geocode=IND)
            if tweets:
                exitcount = 0
            else:
                exitcount += 1
                if exitcount == 3:
                    if search_phrase == search_phrases[-1]:
                        sys.exit('Maximum number of empty tweet strings reached - exiting')
                    else:
                        print('Maximum number of empty tweet strings reached - breaking')
                        break

if __name__ == "__main__":
    main()

SyntaxError: invalid syntax (<ipython-input-6-04a49f5d54fe>, line 81)